In [0]:
%run /eReport/PkgGeneral/PrcGeneral

In [0]:
%run "/eReport/PkgRcd/Cálculo de Provisiones"

In [0]:
import pandas as pd
from datetime import datetime
from datetime import timedelta

class PrcRcdClasificacionAlineada():
  
  def __init__(self, id_solicitud):
      self.id_solicitud = id_solicitud
      
      #Obteniendo parametros
      parametros = PrcGeneral(id_solicitud = self.id_solicitud).obtenerParametros()
      self.empresa = parametros['empresa']  
      self.fecha_proceso = parametros['fecha_proceso']  
      self.fecha_proceso_str = self.fecha_proceso.strftime('%Y-%m-%d') 
      
      #Fecha de actualizacion
      fecha_actualizacion = datetime.now()
      self.fecha_actualizacion_str = fecha_actualizacion.strftime("%Y-%m-%d %H:%M:%S")
      
      #Conexion
      connection = PrcGeneral().databaseConnect()
      self.connection = connection
      
      cursor = self.connection.cursor()
      self.cursor = cursor
      
  def ProcesaClasificacionAlineada(self):
    fechaProcesoRCC = self.fecha_proceso.replace(day=1) - timedelta(days=1)
    fechaProcesoRCC_str = fechaProcesoRCC.strftime('%Y-%m-%d')
    
    cliente = """SELECT DISTINCT A.ID CLIENTE_DEUDOR, B.ID CLIENTE, E.CODIGO_EMPRESA_SBS, D.TIPO_DEUDOR, A.CODIGO_DEUDOR_SBS, A.TIPO_DOCUMENTO_TRIBUTARIO, A.DOCUMENTO_TRIBUTARIO, A.TIPO_DOCUMENTO_IDENTIDAD, A.DOCUMENTO_IDENTIDAD, A.TIPO_PERSONA
    FROM CLIENTE_DEUDOR A 
    JOIN CLIENTE B ON A.ID = B.CLIENTE_DEUDOR
    JOIN CLIENTE_PRODUCTO C ON B.ID = C.CLIENTE
    JOIN CLIENTE_PRODUCTO_CONCEPTO F ON C.ID = F.CLIENTE_PRODUCTO
    JOIN TIPO_CREDITO_SBS D ON D.ID = C.TIPO_CREDITO_SBS
    JOIN EMPRESA E ON E.ID = A.EMPRESA
    WHERE B.FECHA_PROCESO = """"'"+ self.fecha_proceso_str +"""'
    AND A.EMPRESA = """"'"+ str(self.empresa) +"""' 
    AND C.SITUACION_CREDITO = 'N'
    AND F.CONCEPTO_SALDO = 38
    """
    sql_cliente = pd.read_sql_query(cliente, self.connection)
    df_cliente = pd.DataFrame(sql_cliente)
    
    rcc = """SELECT B.ID RCC, D.ID RCC_SALDOS, B.CODIGO_DEUDOR_SBS, B.TIPO_DOCUMENTO_IDENTIDAD, B.DOCUMENTO_IDENTIDAD, B.TIPO_PERSONA, B.TIPO_DOCUMENTO_TRIBUTARIO, B.DOCUMENTO_TRIBUTARIO, D.EMPRESA, D.CUENTA_CONTABLE, D.DIAS_ATRASO, D.IMPORTE, D.CLASIFICACION_DEUDOR
    FROM RCC B
    JOIN RCC_SALDOS D 
    ON B.FECHA_PROCESO = """"'"+ fechaProcesoRCC_str +"""'
    AND  B.ID = D.RCC
    """
    sql_rcc = pd.read_sql_query(rcc, self.connection)
    df_rcc = pd.DataFrame(sql_rcc)
    
    #Obtenemos los saldos por codigo deudor sbs
    ClienteRccSaldos = pd.merge(df_cliente,df_rcc, on = ['CODIGO_DEUDOR_SBS'], how='inner')
    ClienteRccSaldos1 = pd.DataFrame(ClienteRccSaldos)
    ClienteRccSaldos1.drop(['TIPO_DOCUMENTO_IDENTIDAD_y','DOCUMENTO_IDENTIDAD_y', 'TIPO_PERSONA_y', 'TIPO_DOCUMENTO_TRIBUTARIO_y', 'DOCUMENTO_TRIBUTARIO_y'], axis="columns", inplace=True) 
    ClienteRccSaldos1.rename(columns={'TIPO_DOCUMENTO_TRIBUTARIO_x':'TIPO_DOCUMENTO_TRIBUTARIO',
                                    'DOCUMENTO_TRIBUTARIO_x':'DOCUMENTO_TRIBUTARIO',
                                    'TIPO_DOCUMENTO_IDENTIDAD_x':'TIPO_DOCUMENTO_IDENTIDAD',
                                    'DOCUMENTO_IDENTIDAD_x':'DOCUMENTO_IDENTIDAD',
                                    'TIPO_PERSONA_x':'TIPO_PERSONA'}, inplace=True)
    rfClienteRccSaldos1 = pd.DataFrame(ClienteRccSaldos1)
    
    #Obtenemos los saldos por documento de identidad
    ClienteRccSaldos2 = pd.merge(df_cliente,df_rcc, on = ['TIPO_DOCUMENTO_IDENTIDAD', 'DOCUMENTO_IDENTIDAD','TIPO_PERSONA'], how='inner')
    ClienteRccSaldos2.drop(['CODIGO_DEUDOR_SBS_y','TIPO_DOCUMENTO_TRIBUTARIO_y', 'DOCUMENTO_TRIBUTARIO_y'], axis="columns", inplace=True) 
    ClienteRccSaldos2.rename(columns={'CODIGO_DEUDOR_SBS_x':'CODIGO_DEUDOR_SBS',
                                    'TIPO_DOCUMENTO_TRIBUTARIO_x':'TIPO_DOCUMENTO_TRIBUTARIO',
                                    'DOCUMENTO_TRIBUTARIO_x':'DOCUMENTO_TRIBUTARIO'}, inplace=True)
    
    #Obtenemos los saldos por documento tributario
    ClienteRccSaldos3 = pd.merge(df_cliente,df_rcc, on = ['TIPO_DOCUMENTO_TRIBUTARIO', 'DOCUMENTO_TRIBUTARIO'], how='inner')
    ClienteRccSaldos3.drop(['CODIGO_DEUDOR_SBS_y','TIPO_DOCUMENTO_IDENTIDAD_y','DOCUMENTO_IDENTIDAD_y', 'TIPO_PERSONA_y'], axis="columns", inplace=True) 
    ClienteRccSaldos3.rename(columns={'CODIGO_DEUDOR_SBS_x':'CODIGO_DEUDOR_SBS',
                                    'TIPO_DOCUMENTO_IDENTIDAD_x':'TIPO_DOCUMENTO_IDENTIDAD',
                                    'DOCUMENTO_IDENTIDAD_x':'DOCUMENTO_IDENTIDAD',
                                    'TIPO_PERSONA_x':'TIPO_PERSONA'}, inplace=True)
    
    #unimos las consultas y eliminamos los duplicados
    dfClienteRccSaldos = pd.concat([ClienteRccSaldos1,ClienteRccSaldos2,ClienteRccSaldos3]).drop_duplicates()

    #Obtenemos la cantidad de entidad financieras asociadas a un cliente
    ClienteCantidadEmpresas = dfClienteRccSaldos[['CLIENTE_DEUDOR','RCC','EMPRESA']].drop_duplicates()
    
    #Obtenemos solo los clientes que tenga deuda en mas de 2 entidades
    dfClienteEntidades = ClienteCantidadEmpresas.groupby('RCC')
    dfClienteEntidades = dfClienteEntidades.filter(lambda x: len(x) > 1)
    dfClienteEntidadesx = df.groupby('A').filter(lambda x: len(x)>1).groupby('A').sum()
    print(dfClienteEntidades)
    #Obtenemos el resumen de saldos del cliente
    ##primero obtemos el saldo del cliente con deuda en mas de 2 entidades
    dfResumenSaldos = np.merge(dfClienteRccSaldos, dfClienteEntidades, how='inner')
    print(dfResumenSaldos)
    ##luego filtramos por cuenta contable
    
    #traemos las cuenta contable
    df_cuenta_contable_vigente = PRC_RCD_PROVISIONES(4).cuentaContableVigente(self.fecha_proceso_str)
    df_cuenta_contable_vigente['CUENTA_CONTABLE_LEN'] = df_cuenta_contable_vigente['CUENTA_CONTABLE'].str.len()
    df_cuenta_contable_vigente['CUENTA_CONTABLE'] = df_cuenta_contable_vigente['CUENTA_CONTABLE'].str[:'CUENTA_CONTABLE_LEN']
    
    #unimos los dataframes
    ClienteRccSaldosResumen = pd.merge(dfClienteRccSaldos,df_cuenta_contable_vigente, on = 'CUENTA_CONTABLE', how='inner')


    
    
    
    
#Ejecucion prueba
objeto = PrcRcdClasificacionAlineada(1)
objeto.ProcesaClasificacionAlineada()

CLIENTE_DEUDOR RCC EMPRESA
0 38020 1 0142
1 38020 1 0004

--------------------------------------------------------------------------- 
 AttributeError Traceback (most recent call last)
 <command-3426265308973094> in <module> 
 110 #Ejecucion prueba 
 111 objeto = PrcRcdClasificacionAlineada ( 1 ) 
 --> 112 objeto . ProcesaClasificacionAlineada ( ) 

 <command-3426265308973094> in ProcesaClasificacionAlineada (self) 
 91 #Obtenemos el resumen de saldos del cliente 
 92 ##primero obtemos el saldo del cliente con deuda en mas de 2 entidades 
 ---> 93 dfResumenSaldos = np . merge ( dfClienteRccSaldos , dfClienteEntidades , how = 'inner' ) 
 94 print ( dfResumenSaldos ) 
 95 ##luego filtramos por cuenta contable 

 /databricks/python/lib/python3.8/site-packages/numpy/__init__.py in __getattr__ (attr) 
 212 return Tester
 213 else : 
 --> 214 raise AttributeError("module {!r} has no attribute "
 215 "{!r}".format(__name__, attr))
 216 

 AttributeError : module 'numpy' has no attribute 'merge'